In [3]:
from data_loader import *

In [10]:
PARAMS = {'seed': 1234,
          'batch_size': 1,
          'n_samples': 10,
          'split': [0.7, 0.2, 0.1]}
dataset = Dataset(PARAMS, "train", "cpu")

In [11]:



for (x,z,y) in dataset:
    print(x, z, y)
    break

print(dataset.size_p, dataset.size_v)

tensor([[ 40.7516, -73.9786,  40.7775, -73.9567,  40.7558, -73.9900,  40.7210,
         -73.9787,  40.7272, -74.0074,  40.7500, -73.9951,  40.7245, -74.0019,
          40.7646, -73.9731,  40.7619, -73.9662,  40.7587, -73.9758,  40.7789,
         -73.9452,  40.7926, -73.9753,  40.7527, -73.9895,  40.7702, -73.9574,
          40.7430, -73.9739,  40.7625, -73.9684,  40.7492, -74.0056,  40.7382,
         -74.0064,  40.7639, -73.9834,  40.7635, -73.9851]]) tensor([[ 42.2696, -83.7394,  42.2696, -83.7394,  42.2696, -83.7394,  42.2696,
         -83.7394,  42.2696, -83.7394]]) tensor([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])
10 5


In [107]:
import torch
import torch.nn as nn
import torch.nn.functional as F
Ten = torch.FloatTensor

class Net(nn.Module):
    def __init__(self, size_p, size_v):
        """ There are size_p output layers (one for each passenger) 
            of size size_v. The semantic of each of these outputs
            is to likelihood that passenger p will be in vechile v
        """
        super(Net, self).__init__()
        self._ip = size_p*4
        self._iv = size_v*2
        self._hp = size_p*2
        self._hv = size_v
        self._h = size_v
        self._af = nn.ReLU
        self._of = nn.Sigmoid
        self._passengers = list(range(size_p))

        self.i_layer_P = nn.Sequential(
            nn.Linear(self._ip, self._hp),
            nn.BatchNorm1d(self._hp),
            self._af(inplace=True)
        )
        
        self.i_layer_V = nn.Sequential(
            nn.Linear(self._iv, self._hv),
            nn.BatchNorm1d(self._hv),
            self._af(inplace=True)
        )
        
        self.h_layer = nn.Sequential(
            nn.Linear(self._hp + self._hv, self._h),
            nn.BatchNorm1d(self._h),
            self._af(inplace=True)
        ) 
        
        self.o_layers = {p: nn.Sequential(
            nn.Linear(self._h, size_v),
            #nn.Sigmoid()
            nn.Softmax(dim=1)
            ) for p in self._passengers}
             
    def forward(self, p, v):
        hp = self.i_layer_P(p)
        hv = self.i_layer_V(v)
        hout = self.h_layer(torch.cat((hp, hv),1))
        o = [self.o_layers[p](hout) for p in self._passengers]
        return torch.cat(o,1)
        #return torch.cat(tuple(self.o_layers[p](o) for p in self._passengers))

In [108]:
net = Net(dataset.size_p, dataset.size_v)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.MSELoss()

In [109]:
for e in range(1000):
    for (p, v, y) in dataset:
        y_pred = net(p, v)
        L = loss(y_pred, y)
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
    if e % 10 == 0:
        print(L)

tensor(0.1556, grad_fn=<MseLossBackward>)
tensor(0.1494, grad_fn=<MseLossBackward>)
tensor(0.1476, grad_fn=<MseLossBackward>)
tensor(0.1470, grad_fn=<MseLossBackward>)
tensor(0.1467, grad_fn=<MseLossBackward>)
tensor(0.1464, grad_fn=<MseLossBackward>)
tensor(0.1462, grad_fn=<MseLossBackward>)
tensor(0.1459, grad_fn=<MseLossBackward>)
tensor(0.1457, grad_fn=<MseLossBackward>)
tensor(0.1456, grad_fn=<MseLossBackward>)
tensor(0.1455, grad_fn=<MseLossBackward>)
tensor(0.1456, grad_fn=<MseLossBackward>)
tensor(0.1453, grad_fn=<MseLossBackward>)
tensor(0.1453, grad_fn=<MseLossBackward>)
tensor(0.1453, grad_fn=<MseLossBackward>)
tensor(0.1453, grad_fn=<MseLossBackward>)
tensor(0.1452, grad_fn=<MseLossBackward>)
tensor(0.1451, grad_fn=<MseLossBackward>)
tensor(0.1452, grad_fn=<MseLossBackward>)
tensor(0.1451, grad_fn=<MseLossBackward>)
tensor(0.1450, grad_fn=<MseLossBackward>)
tensor(0.1449, grad_fn=<MseLossBackward>)
tensor(0.1450, grad_fn=<MseLossBackward>)
tensor(0.1449, grad_fn=<MseLossBac

KeyboardInterrupt: 